# Local evaluation of Cloud SQL data

This notebook is for local dev environments to inspect Cloud SQL data using Google Cloud SQL Auth Proxy Client.

## Import Libraries

In [75]:
import numpy as np
import pandas as pd
import sqlalchemy
import plotly.express as px

import os

---

## Functions

The following function creates an sqlalchemy engine to use for connecting to Cloud SQL.

In [ ]:
def connect_tcp_socket() -> sqlalchemy.engine.base.Engine:
    """
    Initializes a TCP connection pool for a Cloud SQL instance of Postgres.
    """
    db_host = "127.0.0.1"
    db_port = "5432"
    db_user = "postgres"
    db_pass = os.environ["gcp_postgres_pw"]
    db_name = os.environ["gcp_db_name"]

    pool = sqlalchemy.create_engine(
        # Equivalent URL:
        # postgresql+pg8000://<db_user>:<db_pass>@<db_host>:<db_port>/<db_name>
        sqlalchemy.engine.url.URL.create(
            drivername="postgresql+pg8000",
            username=db_user,
            password=db_pass,
            host=db_host,
            port=db_port,
            database=db_name,
        ),
        # ...
    )
    return pool


The following function queries a specified table from AusbildungMining's Cloud SQL and returns all content as pandas DataFrame:

In [76]:
engine = connect_tcp_socket()

def execute_sql_query(table='beruf'):
    with engine.connect() as conn:
        # Construct your SQL query using the 'text' function
        sql = sqlalchemy.text(f"""SELECT * FROM "ArbeitsagenturMining".arbeit_{table}""")

        # Execute the query with parameters (recommended for security)
        result = conn.execute(sql)

        # Fetch the results
        rows = result.fetchall()

        return pd.DataFrame(rows)

---

## Single Query

The following blocks perform a single query on one table.

It ensures a working connection and enables a quick look into the data.

In [77]:
param_select = 'beruf'

In [78]:
df = execute_sql_query(param_select)
df

,id,timestamp,bundesland,beruf,stellen
0,1,2025-03-04 19:23:16.264654,Schleswig-Holstein,Altenpflegehelfer/in,224
1,2,2025-03-04 19:23:16.264654,Schleswig-Holstein,Altenpfleger/in,307
2,3,2025-03-04 19:23:16.264654,Schleswig-Holstein,Anlagenmechaniker/in,60
3,4,2025-03-04 19:23:16.264654,Schleswig-Holstein,"Anlagenmechaniker/in - Sanitaer-, Heizungs- un...",226
4,5,2025-03-04 19:23:16.264654,Schleswig-Holstein,Auslieferungsfahrer/in (nicht Verkaufsfahrer/in),75
...,...,...,...,...,...
53295,53296,2025-05-06 01:00:41.223602,Thüringen,Tischler/in,126
53296,53297,2025-05-06 01:00:41.223602,Thüringen,Verkaeufer/in,587
53297,53298,2025-05-06 01:00:41.223602,Thüringen,Werkzeugmechaniker/in,63
53298,53299,2025-05-06 01:00:41.223602,Thüringen,Zahnmedizinische/r Fachangestellte/r,104


In [79]:
df_temp = df.copy().groupby(['timestamp',param_select],as_index=False)['stellen'].mean()

professions = pd.Series(df_temp[param_select].unique()).sample(10)

df_temp = df_temp[df_temp[param_select].isin(professions)]

px.line(df_temp,
        x='timestamp',
        y='stellen',
        color=param_select)

---

## Multiple queries / store

To query all tables and store them as parquet files, the following blocks are used:

In [80]:
# Suffix for all tables
params = ['arbeitgeber',
          'arbeitsort_plz',
          'arbeitszeit',
          'befristung',
          'beruf',
          'branche'
          ]

In [ ]:
# Go through all tables, query them and save the content as parquet-files
for param_select in params:
    # Query data
    df = execute_sql_query(param_select)
    # Define filepath to save
    filepath = f"../frontend_streamlit/data/AusbildungMining/ArbeitsagenturMining_Arbeit_{param_select}_2025-05-05.parquet"
    # Save table as parquet
    df.to_parquet(filepath)